In [0]:
import csv
import pandas
headers=['id','tweet','subtask_a']
data = pandas.read_csv("Greek/offenseval-greek-training-v1.tsv", delimiter='\t',names=headers)
print(data.shape)

(8744, 3)


In [0]:
data=data[1:]
print(data.head())


     id                                              tweet subtask_a
1  1172  @USER Οι μουσουλμάνες που τις βιάζουν έτσι κ α...       OFF
2  4078  Η Κάτια προσπαθεί να πείσει οτι δεν είναι ελέφ...       NOT
3   135  Καλά γιατί λες ότι, είσαι νέος αφού γεννήθηκες...       NOT
4  9056          Με Φατσεα ξεκινησαμε...... #Kokkinopotami       NOT
5  5344                  #gntmgr Κάτια πόσο γλυκιά, εμετός       NOT


In [0]:
print(data.shape)

(8743, 3)


In [0]:
import numpy as np
#https://towardsdatascience.com/linear-regression-in-6-lines-of-python-5e1d0cd05b8d
#https://stackoverflow.com/questions/13187778/convert-pandas-dataframe-to-numpy-array
dfnumpy=data.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y=dfnumpy[:, 2].reshape(-1, 1)


In [0]:
print(y.shape)


(8743, 1)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.33, random_state=42)



In [0]:
print(X_train.shape)
print(x.shape)
print(y_train.shape)
print(y_test.shape)

(5857, 1)
(8743, 1)
(5857, 1)
(2886, 1)


In [0]:
arrt=x[:,0]
#TOOD
#arrt=[lamda m: for m in arrt re.sub(r' #', 'HASHTAG', arrt)]

In [0]:
print(arrt.shape)

(8743,)


In [0]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
allTokens=[];

In [0]:
import string


#LEMMATIZATION
import spacy
import el_core_news_sm 
from spacy.tokenizer import Tokenizer

nlp=el_core_news_sm.load()

In [0]:
def preprocess(arrt):
    allTokens=[]
    noises = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m',"’"]
    stopwords =[]
    for txt in arrt:
        x1=tknzr.tokenize(txt)
        x2=list(filter(lambda t: t not in string.punctuation and t not in noises ,x1)
        #print(x1)
        #if len(x2)!=0:dont remove 0 list as issue in svm training
        allTokens.append(x2)
    return allTokens

allTokens=preprocess(arrt)
print(arrt.shape)
print(allTokens)

SyntaxError: invalid syntax (<ipython-input-13-93df5f904e73>, line 10)

In [0]:
def preprocess(arrt):
    noises = ['tweet','URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m',"’"]
    allTokens =[]
    for txt in arrt:
        x1=tknzr.tokenize(txt)
        x2=[]
        for t in x1:
            if t not in string.punctuation and t not in noises:
                x2.append(t)
                #x2=list(filter(lambda t: t not in string.punctuation and t not in noises ,x1)
                
        #if len(x2)!=0 :removed as y needs x
        allTokens.append(list(x2))
               
    return allTokens

allTokens=preprocess(arrt)
print(arrt.shape)
print(len(allTokens))

print(len(allTokens[0:8744]))

In [0]:
#TOKENIZATION USING SPACY 
# BUT NOT TWEET TOEKNIZED
# too time consuming
'''
allTokens1=[]
for token in x:
    x2=[]
    t=nlp(token[0])
    for t2 in t:
        t1=str(t2)
        if t1 not in string.punctuation and t1 not in noises:
            x2.append(t2.lemma_)
    allTokens1.append(x2)
'''

In [0]:
print(allTokens[0:4])
print(string.punctuation)

In [0]:
#print(allTokens1[0:4])

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None) 


In [0]:
tfidf.fit(allTokens)

In [0]:
tfidf.vocabulary_

In [0]:
allTokenstrain=preprocess(X_train[:,0])
allTokenstest=preprocess(X_test[:,0])

Train_X_Tfidf = tfidf.transform(allTokenstrain)
Test_X_Tfidf = tfidf.transform(allTokenstest)

In [0]:
print(Train_X_Tfidf.shape)
print(Test_X_Tfidf.shape)
print(len(allTokenstrain))

In [0]:

from sklearn.svm import SVC
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

#y_train1[y_train=="OFF" ] = 1 
#y_train1[y_train=="NOT"]=0

#y_test[y_test=="OFF" ] = 1 
#y_test[y_test=="NOT"]=0
#Y_train=y_train.flatten()
#Y_test=y_test.flatten()
#print(Train_X_Tfidf.shape)
print(y_train.shape)
print(y_test.shape)
print(Train_X_Tfidf.shape)
yTrain=le.fit_transform(y_train.flatten())
print(yTrain.shape)
print(le.classes_)
yTest=le.fit_transform(y_test.flatten())

print(le.classes_)


In [0]:
#checking SVM 
#code has been inspired from 
#https://github.com/ahmedhammad97/Offensive-Language-Detection

In [0]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
SVM = SVC(class_weight='balanced')
SVM=GridSearchCV(SVM,parameters)
SVM.fit(Train_X_Tfidf,yTrain)
SVM=SVM.best_estimator_

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

predictions_SVM = SVM.predict(Test_X_Tfidf)

print("SVM f1 score -> ",f1_score(predictions_SVM, yTest))
print("SVM accuracy score",accuracy_score(predictions_SVM,yTest )*100)


In [0]:
#checking random forest classifier 
#code has been inspired from 
#https://github.com/ahmedhammad97/Offensive-Language-Detection

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
classifier = RandomForestClassifier(max_depth=800, min_samples_split=5)
params = {'n_estimators': [n for n in range(50,200,50)], 'criterion':['gini','entropy'], }
classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
classifier.fit(Train_X_Tfidf,yTrain)
classifier = classifier.best_estimator_

In [0]:
test_predictions = classifier.predict(Test_X_Tfidf)

print("Random Forest f1 score -> ",f1_score(test_predictions, yTest))
print("Random Forest score>",accuracy_score(test_predictions,yTest )*100)

In [0]:

from sklearn.naive_bayes import GaussianNB, MultinomialNB
classifier = MultinomialNB(alpha=0.7)
classifier.fit(Train_X_Tfidf,yTrain)


In [0]:
test_predictions = classifier.predict(Test_X_Tfidf)

print("Naive Bayes f1 score -> ",f1_score(test_predictions, yTest))
print("Naive Bayes score>",accuracy_score(test_predictions,yTest )*100)

In [0]:
#checking Logistic Regression 
#code has been inspired from 
#https://github.com/ahmedhammad97/Offensive-Language-Detection

In [0]:
from sklearn.linear_model import LogisticRegression
clflr = LogisticRegression(multi_class='auto', solver='newton-cg',class_weight = 'balanced')
classifier = GridSearchCV(classifier, {"C":np.logspace(-3,3,7), "penalty":["l2"]}, cv=3, n_jobs=4)
classifier.fit(Train_X_Tfidf,yTrain)
classifier = classifier.best_estimator_

In [0]:
test_predictions = classifier.predict(Test_X_Tfidf)

print("Logistic Regression f1 score -> ",f1_score(test_predictions, yTest))
print("Logistic Regression score>",accuracy_score(test_predictions,yTest )*100)

In [0]:
#checking Knearest Negihbours
#code has been inspired from 
#https://github.com/ahmedhammad97/Offensive-Language-Detection

In [0]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_jobs=4)
params = {'n_neighbors': [3,5,7,9], 'weights':['uniform', 'distance']}
classifier = GridSearchCV(classifier, params, cv=3, n_jobs=4)
classifier.fit(Train_X_Tfidf,yTrain)
classifier = classifier.best_estimator_

In [0]:
test_predictions = classifier.predict(Test_X_Tfidf)

print("K nearest Neighbours f1 score -> ",f1_score(test_predictions, yTest))
print("K nearest Neighbours score>",accuracy_score(test_predictions,yTest )*100)